In [310]:
import sys
sys.path.append('../core')
%matplotlib inline

In [311]:
import torch
torch.manual_seed(0)
import numpy as np
import torch.optim as optim
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [312]:
import dojo
from display import*
# from mnist import load_mnist
# device, tra_X, tra_y, tes_X, tes_y = load_mnist(False)
# def unique(x, dim=-1):
#     unique, inverse = torch.unique(x, return_inverse=True, dim=dim)
#     perm = torch.arange(inverse.size(dim), dtype=inverse.dtype, device=inverse.device)
#     inverse, perm = inverse.flip([dim]), perm.flip([dim])
#     return unique, inverse.new_empty(unique.size(dim)).scatter_(dim, inverse, perm)

# un,ids = unique(tra_y,dim=0)
# ids = ids[:1]
# # print(len(ids))
# train_X = tra_y[ids]
# train_y = tra_X[ids]
# test_X = tra_y[ids]
# test_y = tra_X[ids]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
res=28
n = 10000
# positioning
xr = torch.range(0,res-1,device=device) - (res-1)/2
xi,yi = torch.meshgrid(xr,xr, indexing='ij')
x = torch.stack((xi,yi),dim=2)
# data
y = torch.stack((x[:,:,0],x[:,:,1]),dim=2).unsqueeze(2) # res x res x 1 x 2 #,x[:,:,0]**2/2,x[:,:,1]**2/2
rc = torch.randn(size=(1,1,2,n),device=device)
y = torch.matmul(y,rc).T.squeeze().reshape(n,res**2) # n x im_size
# normalize
y /= res**2
y = (y - y.mean(dim=1,keepdim=True))#/y.std(dim=1,keepdim=True)
# to device (and 2D)
data = y.to(device)

train_X = data[:7500]
train_y = data[:7500]
test_X = data[7500:]
test_y = data[7500:]

(y[0].detach().cpu().numpy().reshape(res,res))


array([[ -1.34830205e-02,  -1.46624371e-02,  -1.58418529e-02,
         -1.70212686e-02,  -1.82006862e-02,  -1.93801019e-02,
         -2.05595195e-02,  -2.17389353e-02,  -2.29183510e-02,
         -2.40977686e-02,  -2.52771843e-02,  -2.64565982e-02,
         -2.76360158e-02,  -2.88154315e-02,  -2.99948491e-02,
         -3.11742648e-02,  -3.23536843e-02,  -3.35330963e-02,
         -3.47125120e-02,  -3.58919315e-02,  -3.70713472e-02,
         -3.82507630e-02,  -3.94301787e-02,  -4.06095982e-02,
         -4.17890139e-02,  -4.29684296e-02,  -4.41478454e-02,
         -4.53272648e-02],
       [ -1.13048600e-02,  -1.24842767e-02,  -1.36636933e-02,
         -1.48431081e-02,  -1.60225257e-02,  -1.72019415e-02,
         -1.83813572e-02,  -1.95607748e-02,  -2.07401905e-02,
         -2.19196081e-02,  -2.30990238e-02,  -2.42784414e-02,
         -2.54578553e-02,  -2.66372710e-02,  -2.78166886e-02,
         -2.89961044e-02,  -3.01755201e-02,  -3.13549377e-02,
         -3.25343534e-02,  -3.37137692e-02,

In [313]:
from DNN import DNN

In [314]:
def LOSS(yhat,y):
    e0 = torch.nn.MSELoss()(yhat,y)
    p1 = torch.nn.functional.softmax(yhat,dim=1)
    p2 = torch.nn.functional.softmax(y,dim=1)
    e1 = 50*torch.nn.KLDivLoss()(p1,p2)
    return e0 + e1

def make_x(p):
    b = p.shape[0]
    xr = torch.range(0,res-1,device=device)/res - 0.5
    xi,yi = torch.meshgrid(xr,xr, indexing='ij')
    return torch.stack((xi,yi),dim=2).unsqueeze(0).expand(b,res,res,2)

def reshape_p(p):
    d = 2
    nv = 6
    b = p.shape[0]
    n_func = p.shape[1]//(nv)
    return torch.reshape(p,(b,nv,n_func)), n_func

def gaussian(x, mu=0, sigma=1):
    return torch.exp(-((x - mu)**2 / (2 * sigma**2))) / (sigma * np.sqrt(2 * np.pi))

def hat(x):
    p = torch.sign(x) 
    return (1-torch.nn.functional.relu(p *x)) * (torch.trunc(x)==0)

def forward(x,p):
    d=2 # hard coded for now
    b = p.shape[0]
    p_, n_func = reshape_p(p)
    x_ = torch.reshape(x,(b,np.prod(x.size()[1:-1]),d))
    # batched mul of every x in x_ with every p in p_[:2,]
    # print(torch.bmm(x_,p_[:,:d,:]).size())
    # print(torch.bmm(x_[:,:,0].unsqueeze(2),torch.cos(p_[:,0,:]).unsqueeze(1)).size())

    # x :batch, image_size, n_func
    # p :batch, 3, n_func
    # pre-act :batch, image_size, n_func, 2
    act = lambda x : hat(x[:,:,:,0])*hat(x[:,:,:,1])#torch.nn.SiLU()
    # act :batch, image_size, n_func
    angle = 2*torch.pi * torch.frac(p_[:,1,:] + 0.5)
    c = torch.cos(angle).unsqueeze(1)
    s = torch.sin(angle).unsqueeze(1)
    xs = x_[:,:,0].unsqueeze(2)
    ys = x_[:,:,1].unsqueeze(2)
    xscale = p_[:,2,:].unsqueeze(1).expand(b,x_.shape[1],n_func)
    yscale = p_[:,3,:].unsqueeze(1).expand(b,x_.shape[1],n_func)
    xshift = p_[:,4,:].unsqueeze(1)
    yshift = p_[:,5,:].unsqueeze(1)
    a = act( \
                torch.stack(( \
                    torch.mul((torch.bmm(xs,c) - torch.bmm(ys,s)),xscale) + xshift, \
                    torch.mul((torch.bmm(xs,s) + torch.bmm(ys,c)),yscale) + yshift, \
                ),dim=3) \
            )
    return torch.bmm((1/n_func + p_[:,0,:]).unsqueeze(1),a.mT)

def constraint(x,p,size0):
    if p is None:
        return torch.zeros(size0,device=device)
    p_ = reshape_p(p)[0]
    return torch.mean(torch.concatenate((p_[:,0,:]*0.0,torch.trunc(p_[:,1,:])**2,*[p_[:,j,:]*0.0 for j in range(2,6)]),dim=1),dim=0).expand(size0)


def criterion(p, y):
    # n_func = p.shape[1]//(3)
    out = forward(make_x(p),p).reshape(p.shape[0],res**2)
    # print(out.mean())
    p_ = reshape_p(p)[0]
    return LOSS(out, y) + torch.mean(torch.trunc(p_[:,1,:]+0.5)**2)


In [315]:
hat(torch.Tensor([-5,-4,-3,-2,-1,0,1,2,3,4,5])/3)

tensor([-0.0000, -0.0000, 0.0000, 0.3333, 0.6667, 1.0000, 0.6667, 0.3333, 0.0000,
        -0.0000, -0.0000])

In [316]:
D = dojo.dojo()
D.optimizer = lambda x : optim.ASGD(x, lr=0.0001, weight_decay=1e-8) # optim.Adam(net.parameters(), lr=base_lr)

In [317]:
n_param=120 # a multiple of 6
net = DNN(sizes=[n_param + res**2,2*(n_param + res**2),n_param,n_param],d2_input=False,classification=False,activation = torch.nn.SiLU(),criterion=criterion,bias=False, init=np.sqrt(2.0))
net.to(device)


DNN(
  (activation): SiLU()
  (prob): Identity()
  (layers): ModuleList(
    (0): Linear(in_features=904, out_features=1808, bias=False)
    (1): Linear(in_features=1808, out_features=120, bias=False)
    (2): Linear(in_features=120, out_features=120, bias=False)
  )
)

In [307]:
D.max_batch_size=train_X.size(0)
D.epochs=100
# torch.autograd.set_detect_anomaly(True)
report = D.train(net, train_X, train_y, clip=10, constraint=constraint,start_batch_size=100)
ecran(net, test_X, test_y, criterion, report, classification=False)

 86%|████████▌ | 86/100 [00:54<00:08,  1.58it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 388.00 MiB (GPU 0; 8.00 GiB total capacity; 6.09 GiB already allocated; 0 bytes free; 6.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
report[-1]

In [ ]:
test_X2 = torch.concatenate((test_X,torch.zeros((test_X.size(0),n_param),device=device)),dim=1)

p = net(test_X2)
outs = forward(make_x(p),p)

# plt.plot(report[-1])
ni=5
si=2
fig,axs=plt.subplots(1,5)
[axs[i].imshow(outs[si+i].detach().cpu().numpy().reshape(res,res)) for i in range(ni)]
[axs[i].axis('off') for i in range(ni)]
fig,axs=plt.subplots(1,5)
[axs[i].imshow(test_y[si+i].detach().cpu().numpy().reshape(res,res)) for i in range(ni)]
[axs[i].axis('off') for i in range(ni)]
plt.figure()
plt.plot(reshape_p(p)[0][:,1,:].detach().cpu().numpy())
plt.show()

In [ ]:
from torchviz import *
def resize_graph(dot, size_per_element=0.15, min_size=12):
    """Resize the graph according to how much content it contains.
    Modify the graph in place.
    """
    # Get the approximate number of nodes and edges
    num_rows = len(dot.body)
    content_size = num_rows * size_per_element
    size = max(min_size, content_size)
    size_str = str(size) + "," + str(size)
    dot.graph_attr.update(size=size_str)


dot = make_dot(outs.mean(), params=dict(net.named_parameters()), show_attrs=True)
resize_graph(dot,size_per_element=0.15, min_size=24)
dot.render("graph", format="svg")

In [ ]:
mask = (torch.rand((28,28)) > 0.5).to(device)
# plt.imshow(train_X[0].detach().cpu().numpy())
mixed = train_X[234]*mask + train_X[234]*(~mask)
plt.imshow(mixed.detach().cpu().numpy())

In [320]:
# del net, report
torch.cuda.empty_cache()